In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download e exploração inicial dos dados

In [ ]:
!git clone https://github.com/pubmedqa/pubmedqa.git

import json

file_path = 'pubmedqa/data/ori_pqal.json'

with open(file_path, 'r') as f:
    data = json.load(f)

sample_key = list(data.keys())[0]
print(f"\nCampos disponíveis: {list(data[sample_key].keys())}\n")

print("=" * 60)
print("Exploração de dados - PubMedQA")
print("=" * 60)

for i, key in enumerate(list(data.keys())[:3]):
    item = data[key]

    print(f"\nExemplo {i+1} | ID: {key}")
    print("-" * 60)
    print(f"Question: {item.get('QUESTION', 'N/A')}")

    context = " ".join(item.get('CONTEXTS', []))
    print(f"Context: {context[:300]}...")

    print(f"Labels: {item.get('LABELS', 'N/A')}")
    print(f"Decision: {item.get('final_decision', 'N/A')}")
    print(f"Answer: {item.get('LONG_ANSWER', 'N/A')[:200]}...")
    print(f"Meshes: {item.get('MESHES', 'N/A')}")
    print(f"Year: {item.get('YEAR', 'N/A')}")
    print(f"Reasoning required pred: {item.get('reasoning_required_pred', 'N/A')}")
    print(f"Reasoning free pred: {item.get('reasoning_free_pred', 'N/A')}")

print(f"\n\nTotal de registros: {len(data)}")

# Pré-processamento

In [ ]:
import json
import re
import unicodedata
import os

def preprocess_text(text):
    """Normaliza e limpa texto"""
    if not isinstance(text, str):
        return ""

    # Normalização unicode
    text = unicodedata.normalize('NFKC', text)

    # Normalização de espaços
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def map_decision(decision):
    decision = decision.lower()
    if decision == "yes":
        return "SIM"
    elif decision == "no":
        return "NÃO"
    elif decision == "maybe":
        return "TALVEZ"
    return

def sanitize_answer(text):
    forbiden_list = [
        "assistant.",
        "assistant.Decision",
        "assistant.Undefinitions",
        "definitions",
        "context",
        "analysis"
    ]
    for b in forbiden_list:
        text = text.replace(b, "")
    return text.strip()

def preprocess_dataset(input_path, output_path):
    """Pré-processa o dataset original completo"""
    print(f"Carregando {input_path}...")
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = {}

    for key, item in data.items():

        # QUESTION
        question = preprocess_text(item.get('QUESTION', ''))

        # CONTEXTS (string ou lista)
        context_raw = " ".join(item.get('CONTEXTS', []))
        context = preprocess_text(context_raw)

        # DECISION
        decision_raw = item.get('final_decision', 'N/A')
        decision = map_decision(decision_raw)

        # LONG_ANSWER
        long_answer = preprocess_text(item.get('LONG_ANSWER', ''))
        long_answer = sanitize_answer(long_answer)

        answer = f"Decisão: {decision}\nJustificativa: {long_answer}"

        processed_data[key] = {
            "QUESTION": question,
            "CONTEXTS": context,
            "FINAL_ANSWER": answer,
            "YEAR": item.get('YEAR', 'N/A')
        }

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(processed_data, f_out, indent=4, ensure_ascii=False)

    print(f"✓ Processados {len(processed_data)} registros")
    return len(processed_data)

# Processar dataset original completo
total = preprocess_dataset(
    'pubmedqa/data/ori_pqal.json',
    'data_processed/ori_pqal_preprocessed.json'
)

print(f"\nPré-processamento concluído: {total} registros")
print("Arquivo salvo em: data_processed/ori_pqal_preprocessed.json")

# Anonimização

In [ ]:
import json
import re
import os

def anonymize_text(text):
    """Remove dados sensíveis (LGPD/HIPAA compliance)"""
    if not isinstance(text, str):
        return ""

    text = re.sub(r'(Dr\.|Dra\.|Doctor|Prof\.|MD)\s+[A-Z][a-z]+(\s+[A-Z][a-z]+)?', '[NOME]', text)
    text = re.sub(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', '[EMAIL]', text)
    locations = r'(Israel|Denmark|Chile|Texas|France|United Kingdom|UK|USA|Pakistan|Karachi|Jordan|Japan|Australia|North Carolina|Washington)'
    text = re.sub(locations, '[LOCAL]', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{6,}\b', '[ID]', text)
    text = re.sub(r'\b(19|20)\d{2}\b', '[ANO]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)

    return text

def anonymize_dataset(input_path, output_path):
    """Anonimiza o dataset pré-processado completo"""
    print(f"Carregando {input_path}...")
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    anonymized = {}

    for key, item in data.items():
        new_id = f"HOSP_REG_{key[:4]}"

        anonymized[new_id] = {
            "QUESTION": anonymize_text(item.get('QUESTION', '')),
            "CONTEXTS": anonymize_text(item.get('CONTEXTS', '')),
            "FINAL_ANSWER": anonymize_text(item.get('FINAL_ANSWER', '')),
            "YEAR": item.get('YEAR', 'N/A')
        }

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(anonymized, f_out, indent=4, ensure_ascii=False)

    print(f"✓ Anonimizados {len(anonymized)} registros")
    return len(anonymized)

# Anonimizar dataset pré-processado
total = anonymize_dataset(
    'data_processed/ori_pqal_preprocessed.json',
    'data_anonymized/ori_pqal_anonymized.json'
)

print(f"\nAnonimização concluída: {total} registros")
print("Arquivo salvo em: data_anonymized/ori_pqal_anonymized.json")

# Exemplo de dado anonimizado
with open('data_anonymized/ori_pqal_anonymized.json', 'r', encoding='utf-8') as f:
    sample = json.load(f)
    first_key = list(sample.keys())[0]
    print(f"\n{'='*60}")
    print("Exemplo de dado anonimizado:")
    print(f"{'='*60}")
    print(f"ID: {first_key}")
    print(f"Question: {sample[first_key]['QUESTION'][:100]}...")

## Análise de Qualidade

In [ ]:
import json
import os
from collections import Counter

def analyze_quality(data):
    issues = {
        'question_vazia': [],
        'context_vazio': [],
        'answer_vazia': [],
        'answer_muito_curta': [],
        'context_muito_curto': []
    }

    for key, item in data.items():
        if not item.get('QUESTION', '').strip():
            issues['question_vazia'].append(key)

        if not item.get('CONTEXTS', '').strip():
            issues['context_vazio'].append(key)

        if not item.get('FINAL_ANSWER', '').strip():
            issues['answer_vazia'].append(key)

        if len(item.get('FINAL_ANSWER', '')) < 50:
            issues['answer_muito_curta'].append(key)

        if len(item.get('CONTEXTS', '')) < 100:
            issues['context_muito_curto'].append(key)

    return issues

def extract_decision(answer):
    if answer.startswith("Decisão: SIM"):
        return "SIM"
    if answer.startswith("Decisão: NÃO"):
        return "NÃO"
    if answer.startswith("Decisão: TALVEZ"):
        return "TALVEZ"
    return "UNKNOWN"

def analyze_distribution(data):
    distribution = Counter()

    for key, item in data.items():
        decision = extract_decision(item.get('FINAL_ANSWER',''))
        distribution[decision] += 1

    return distribution

print("Analisando qualidade dos dados...")

# Analisar test set
with open('/content/data_anonymized/ori_pqal_anonymized.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

issues = analyze_quality(test_data)

print("\nProblemas encontrados:")
for issue_type, ids in issues.items():
    if ids:
        print(f"  {issue_type}: {len(ids)} registros")
    else:
        print(f"  {issue_type}: 0")

print("\nDistribuição:")
dist_test = analyze_distribution(test_data)
for cls, count in dist_test.items():
    pct = (count / len(test_data)) * 100
    print(f"  {cls}: {count} ({pct:.1f}%)")

print(f"\n{'='*60}")
print(f"Total de registros: {len(test_data)}")
print(f"{'='*60}")

## Fine Tunning

Instando as dependências

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

Configuração das variáveis do modelo

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
import os
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]


Conversão do dataset para treinamento


In [ ]:
import json
from datasets import Dataset
import os

DATA_PATH = "data_anonymized/ori_pqal_anonymized.json"
OUTPUT_DATA_PATH = "data_final/final_pqal.json"
SYSTEM_PROMPT = """
Você é um assistente médico-científico. Responda exclusivamente em português.

Responda EXCLUSIVAMENTE com base no contexto fornecido.
Não utilize conhecimento externo.

REGRAS OBRIGATÓRIAS:
- A resposta DEVE conter APENAS duas linhas.
- A primeira linha DEVE começar exatamente com:
  "Decisão: SIM", "Decisão: NÃO" ou "Decisão: TALVEZ"
- A segunda linha DEVE começar exatamente com:
  "Justificativa:"

PROIBIÇÕES ABSOLUTAS:
- NÃO inclua rótulos internos, nomes técnicos, marcadores ou palavras como:
  "assistant.", "assistant.Decision", "assistant.Undefinitions", "definitions", "context", "analysis".
- NÃO inclua listas, títulos, explicações adicionais ou texto fora do formato exigido.
- NÃO repita a pergunta.
- NÃO inclua qualquer texto antes ou depois das duas linhas exigidas.
- Não faça recomendação de tratamentos
- Não faça recomendação de medicamentos

Use:
- SIM quando o contexto apoiar claramente a afirmação.
- NÃO quando o contexto claramente a contradizer.
- TALVEZ apenas quando as evidências forem insuficientes, inconclusivas ou conflitantes.
"""

def validate_answer(answer):
    lines = answer.strip().splitlines()
    if not lines[0].startswith("Decisão:"):
        return False
    if not lines[1].startswith("Justificativa:"):
        return False
    if not any(x in lines[0] for x in ["SIM", "NÃO", "TALVEZ"]):
        return False
    return True

with open(DATA_PATH, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = []
for _, item in raw_data.items():
  if not validate_answer(item["FINAL_ANSWER"]):
    continue

  data.append({
      "messages": [
          {"role": "system", "content": SYSTEM_PROMPT},
          {"role": "user", "content": item["QUESTION"]},
          {"role": "assistant", "content": item["FINAL_ANSWER"]},
      ]
  })

formatted_data = Dataset.from_list(data)

print("Novo formato do dataset:")
print(json.dumps(formatted_data[0], indent=2, ensure_ascii=False))

os.makedirs(os.path.dirname(OUTPUT_DATA_PATH), exist_ok=True)
with open(OUTPUT_DATA_PATH, 'w', encoding='utf-8') as output_file:
  json.dump(formatted_data.to_list(), output_file, indent=4)

print("\n")
print("="*60)
print(f"Dataset final salvo em: {OUTPUT_DATA_PATH}")
print("="*60)

## Carregando o modelo "unsloth/llama-3-8b-bnb-4bit"

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for messages in examples["messages"]:
        full_text = ""
        for msg in messages:
            full_text += msg["content"].strip() + "\n"
        texts.append(full_text.strip() + EOS_TOKEN)
    return {"text": texts}

OUTPUT_PATH_DATASET = "data_final/final_pqal.json"

dataset = load_dataset(
    "json",
    data_files=OUTPUT_PATH_DATASET,
    split="train"
)

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=dataset.column_names
)

print("Primeiro exemplo do dataset formatado:")
print(dataset[0]["text"])

In [ ]:
# @title
os.environ["WANDB_DISABLED"] = "true"
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,                   #Desejável aumentar, porém demora muito
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer_stats = trainer.train()

In [ ]:
from google.colab import drive
from dotenv import load_dotenv
from huggingface_hub import login
import os

# Carregar env
ENV_PATH = "/content/drive/MyDrive/token-hf/env"
load_dotenv(ENV_PATH)

HF_TOKEN = os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

HF_REPO = f"{os.getenv("HF_USER_REPO")}/assistente-medico-lora"
model.push_to_hub(HF_REPO)
tokenizer.push_to_hub(HF_REPO)

# TESTE BÁSICO DO MODELO TREINADO

In [ ]:
test_examples = [
    {
        "question": "O uso diário de protetor solar previne o câncer de pele em pessoas com alto risco de desenvolver a doença?",
        "context": "Estudos longitudinais mostraram que o uso regular e consistente de protetor solar com FPS 30 ou superior, em indivíduos com histórico familiar de melanoma ou com múltiplos nevos atípicos, reduziu significativamente a incidência de novos casos de câncer de pele não melanoma e melanoma em comparação com grupos de controle que usavam protetor solar ocasionalmente ou não usavam. A aplicação correta e reaplicação conforme as instruções são cruciais para a eficácia."
    },
    {
        "question": "A dieta cetogênica é recomendada como tratamento primário para hipertensão arterial em todos os pacientes?",
        "context": "A dieta cetogênica tem mostrado resultados promissores na redução da pressão arterial em alguns estudos, especialmente em pacientes com obesidade e resistência à insulina. No entanto, sua segurança e eficácia a longo prazo como tratamento primário para hipertensão em toda a população de pacientes não foram estabelecidas por completo. Diretrizes atuais recomendam uma abordagem individualizada, considerando comorbidades e potencial para efeitos adversos. Alguns estudos indicam que, em pacientes específicos, pode ser uma opção viável sob supervisão médica, enquanto outros alertam para a necessidade de mais pesquisas antes de uma recomendação generalizada."
    },
    {
        "question": "A vacina contra a gripe é eficaz em 100% dos casos para prevenir a infecção pelo vírus influenza?",
        "context": "A eficácia da vacina contra a gripe varia anualmente e depende de diversos fatores, como a correspondência entre as cepas da vacina e as que estão em circulação, e a idade e o estado de saúde do indivíduo vacinado. Em geral, a vacina é eficaz em 40% a 60% na prevenção da infecção pelo vírus influenza. Embora não seja 100% eficaz, ela reduz significativamente o risco de desenvolver a doença e suas complicações graves, incluindo hospitalizações e mortes."
    },
    {
        "question": "O consumo moderado de café está associado a um risco aumentado de doenças cardiovasculares em adultos saudáveis?",
        "context": "Múltiplos estudos observacionais e meta-análises sugerem que o consumo moderado de café (cerca de 3-4 xícaras por dia) não está associado a um risco aumentado de doenças cardiovasculares em adultos saudáveis. Na verdade, alguns estudos indicam uma possível associação com um risco ligeiramente reduzido de certas condições cardíacas, embora mais pesquisas sejam necessárias para estabelecer causalidade. O consumo excessivo, no entanto, pode estar ligado a efeitos adversos em indivíduos sensíveis."
    },
    {
        "question": "O tratamento com antibióticos é sempre necessário para resfriados comuns?",
        "context": "Resfriados comuns são infecções virais das vias aéreas superiores. Antibióticos são medicamentos projetados para combater infecções bacterianas e não têm efeito contra vírus. O uso desnecessário de antibióticos pode levar à resistência a antibióticos, um problema de saúde pública crescente. Portanto, o tratamento com antibióticos não é indicado para resfriados comuns."
    }
]

print(f"Gerados {len(test_examples)} exemplos de teste.")

In [ ]:
FastLanguageModel.for_inference(model)

eos_id = tokenizer.eos_token_id

print("\n--- Executando testes com exemplos gerados ---")

for i, example in enumerate(test_examples):
    question = example["question"]
    context = example["context"]

    prompt = f"""{SYSTEM_PROMPT}

    Pergunta:
    {question}

    Contexto científico:
    {context}

    Resposta:
    """.strip()

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,
        temperature=0.0,
        eos_token_id=eos_id,
        repetition_penalty=1.1,
        use_cache=True
    )

    generated_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    response = tokenizer.decode(
        generated_ids,
        skip_special_tokens=True
    ).strip()

    print(f"\n--- Exemplo {i+1} ---")
    print(f"Pergunta: {question}")
    print("\n")
    print("Resposta do Modelo:")
    print(response)
    print("\n" + "=" * 80)

print("\n--- Testes concluídos ---")
